In [ ]:
from scipy import stats
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import random

import keras
from keras.preprocessing.sequence import pad_sequences

import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
import seaborn as sns
from pylab import rcParams

from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GroupShuffleSplit

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
#download train data
data = drive.CreateFile({'id': 'here provide the file id from link sharing in from google drive'})
data.GetContentFile('data_cleaned.pkl')

file = open('data_cleaned.pkl','rb')
train = pickle.load(file)

## Negative Sampling

#### Negative Sampling

In [ ]:
# get product attributes data
products = drive.CreateFile({'id': 'here provide the file id from link sharing in from google drive'})
products.GetContentFile('product_attributes_data.tsv.gz')
products = pd.read_table('product_attributes_data.tsv.gz')
products.rename(columns={'rpid' : 'product_id'}, inplace=True)

In [ ]:
#merge with transactions
df_concat = pd.merge(train, products, how='left', on=['product_id'])
# create week, year and month
df_concat.loc[df_concat['day'] <= '2016-01-03', 'year'] = '2015' 
df_concat.loc[df_concat['day'] > '2016-01-03', 'year'] = '2016'
df_concat.loc[df_concat['day'] >= '2017-01-02', 'year'] = '2017' 
df_concat['day'] = pd.to_datetime(df_concat['day'])
df_concat['week'] = df_concat['day'].dt.week.astype('str')
df_concat['month'] = df_concat['day'].dt.month.astype('str')
df_concat['yearandmonth'] = df_concat['year'] + df_concat['month']

In [ ]:
# get unit price by dividing by the quantity
df_concat['price'] = df_concat['price'] / df_concat['quantity']

In [ ]:
# remove missing category and subcategory names
df_concat = df_concat[df_concat['category_name'].notna()]
df_concat = df_concat[df_concat['subcategory_name'].notna()]

We first try to sample a negative from the subcategory.

In [ ]:
df_single_sub_categories = df_concat.groupby(['subcategory_name', 'yearandmonth'], as_index=False).agg({'product_id' : 'nunique'})
single_sub_categories = df_single_sub_categories[df_single_sub_categories['product_id'] == 1].subcategory_name.to_list()
df_subcategory_sampling = df_concat[~df_concat['subcategory_name'].isin(single_sub_categories)]
df_subcategory_sampling = df_subcategory_sampling.reset_index()
df_subcategory_sampling = df_subcategory_sampling[['article_text', 'subcategory_name', 'yearandmonth']]
df_subcategory_sampling = df_subcategory_sampling.drop_duplicates()
#merged this info with actual df using subcategory name
df_subcategory_sampling = df_subcategory_sampling[df_subcategory_sampling['subcategory_name'].notna()]
df_subcategory_sampling['subcategory_rival'] = [np.random.choice(df_subcategory_sampling[(df_subcategory_sampling['subcategory_name']==cat) & (df_subcategory_sampling['yearandmonth']==ynm) & (df_subcategory_sampling['article_text']!=iden)]['article_text']) for cat, ynm, iden in zip(df_subcategory_sampling['subcategory_name'], df_subcategory_sampling['yearandmonth'], df_subcategory_sampling['article_text'])]


If the subcategory consist of only a single products, we sample a product from the same category.

In [ ]:
df_single_categories = df_concat.groupby(['category_name', 'yearandmonth'], as_index=False).agg({'product_id' : 'nunique'})
single_categories = df_single_categories[df_single_categories['product_id'] == 1].category_name.to_list()
df_category_sampling = df_concat[~df_concat['category_name'].isin(single_categories)]
df_category_sampling = df_category_sampling.reset_index()
df_category_sampling = df_category_sampling[['article_text', 'category_name', 'yearandmonth']]
df_category_sampling = df_category_sampling.drop_duplicates()
#merged this info with actual df using category name
df_category_sampling = df_category_sampling[df_category_sampling['category_name'].notna()]
df_category_sampling['category_rival'] = [np.random.choice(df_category_sampling[(df_category_sampling['category_name']==cat) & (df_category_sampling['yearandmonth']==ynm) & (df_category_sampling['article_text']!=iden)]['article_text']) for cat, ynm, iden in zip(df_category_sampling['category_name'], df_category_sampling['yearandmonth'], df_category_sampling['article_text'])]


In [ ]:
all_products = df_concat[['article_text', 'yearandmonth']]
all_products = all_products.drop_duplicates()
# get them in a df
all_products = pd.merge(all_products, df_subcategory_sampling, how='left', on=['article_text', 'yearandmonth'])
all_products = pd.merge(all_products, df_category_sampling, how='left', on=['article_text', 'yearandmonth'])

In [ ]:
all_products['rival_product'] = all_products.subcategory_rival.combine_first(all_products.category_rival)
all_products = all_products[['article_text', 'rival_product', 'yearandmonth']]
df_concat = pd.merge(df_concat, all_products, how='left', on=['article_text', 'yearandmonth'])
df_concat = df_concat[df_concat['rival_product'].notna()]
df_concat['product_index'] = df_concat.index

In [ ]:
df_concat['product_id'] = df_concat['product_id'].astype('int')
df_concat = df_concat[['day'	,'basket_hash',	'user_id', 'quantity',	'price',	'user_visit_count', 'week', 'year', 'rival_product', 'product_index', 'article_text']]
df_concat = df_concat.melt(id_vars=['day'	,'basket_hash',	'user_id', 'quantity',	'price',	'user_visit_count', 'week', 'year', 'product_index'], 
        var_name="indicator", 
        value_name="products")
df_concat['bought'] = np.where(df_concat['indicator']== 'article_text', 1, 0)

In [ ]:
product_ids = products[['article_text', 'product_id']]
product_ids.rename(columns={'article_text' : 'products'}, inplace=True)
product_ids = product_ids.drop_duplicates(subset=['products'])
df_concat = pd.merge(df_concat, product_ids, how='left', on=['products'])
df_concat.rename(columns={'products' : 'article_text'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


#### Add prices of negative sampled products

In [ ]:
#use price information from purchased products to fill in not purchased products:
df_concat_sold = df_concat[df_concat['bought'] == 1]
del df_concat_sold['week']

In [ ]:
per_date_product_price = df_concat_sold[['day', 'product_id', 'price', 'year']]
per_date_product_price['day'] = pd.to_datetime(per_date_product_price['day'])
per_date_product_price['week'] = per_date_product_price['day'].dt.week

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate f

In [ ]:
price_per_week = per_date_product_price.groupby(['product_id', 'week', 'year'], as_index=False).agg(
    {'price': lambda x:stats.mode(x)[0]})

week_and_year = per_date_product_price.groupby(['week', 'year'], as_index=False).agg(
    {})
week_and_year['key'] = 0

product_df = pd.DataFrame(df_concat['product_id'].unique())
product_df['key'] = 0
product_df.rename(columns={0: 'product_id'}, inplace=True)

In [ ]:
merge_with_week_year = pd.DataFrame(week_and_year.merge(product_df, how='outer'))
merge_with_week_year.drop(columns=['key'], inplace=True)

In [ ]:
merge_with_week_year['week'] = merge_with_week_year['week'].astype('str')
merge_with_week_year['year'] = merge_with_week_year['year'].astype('str')
merge_with_week_year['product_id'] = merge_with_week_year['product_id'].astype('str')

price_per_week['product_id'] = price_per_week['product_id'].astype('str')
price_per_week['week'] = price_per_week['week'].astype('str')
price_per_week['year'] = price_per_week['year'].astype('str')


In [ ]:
df_prices = pd.merge(merge_with_week_year, price_per_week, how='left', on=['product_id', 'week', 'year'])
df_prices['price2'] = df_prices.groupby(['product_id', 'year'])['price'].transform(lambda x: x.fillna(method = 'ffill'))
df_prices['price2'] = df_prices.groupby(['product_id', 'year'])['price2'].transform(lambda x: x.fillna(method = 'bfill'))
df_prices['price3'] = df_prices.groupby(['product_id'])['price2'].transform(lambda x: x.fillna(method = 'ffill'))
df_prices['price3'] = df_prices.groupby(['product_id'])['price3'].transform(lambda x: x.fillna(method = 'bfill'))

df_prices.drop(['price', 'price2'], axis=1, inplace=True)
df_prices.rename(columns={'price3': 'price'}, inplace=True)

In [ ]:
df_prices.rename(columns={'price' : 'fillingprice'}, inplace=True)
df_concat['product_id'] = df_concat['product_id'].astype('str')
# only change prices for not bought products
df_concat.loc[df_concat['bought'] == 0, 'price'] = 'empty'
df_concat = pd.merge(df_concat, df_prices, how='left', on=['product_id', 'week', 'year'])

df_concat['fillingprice'] = df_concat['fillingprice'].astype('str')
df_concat.loc[df_concat['price'] == 'empty', 'price'] = df_concat['fillingprice']

In [ ]:
products_add = products[['product_id', 'subcategory_name', 'category_name']]
products_add['product_id'] = products_add['product_id'].astype('str')
df_concat = pd.merge(df_concat, products_add, how='left', on=['product_id'])
df_concat.loc[df_concat['bought'] == 0, 'quantity'] = 0
del df_concat['user_visit_count'], df_concat['indicator'], df_concat['fillingprice']
df_concat.rename(columns={'products' : 'article_text'}, inplace=True)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


## Feature Engineering

In [ ]:
train = df_concat[:]

In [ ]:
train['day'] = pd.to_datetime(train['day'])
train['week'] = train['week'].str.zfill(2)
train["weekandyear"] = train["year"] + train["week"]

#### Label Encoding

In [ ]:
train['product_id_notenc'] = train['product_id']
train['day_notenc'] = train['day']

In [ ]:
# creating label encoders for categorical features

le = LabelEncoder()
le.fit(train['product_id'])
train['product_id'] = le.transform(train['product_id'])

le = LabelEncoder()
le.fit(train['user_id'])
train['user_id'] = le.transform(train['user_id'])

le_day = LabelEncoder()
le_day.fit(train['day'])
train['day'] = le_day.transform(train['day'])

le_day = LabelEncoder()
le_day.fit(train['basket_hash'])
train['basket_hash'] = le_day.transform(train['basket_hash'])

le_sub = LabelEncoder()
le_sub.fit(train['subcategory_name'])
train['subcategory_name'] = le_sub.transform(train['subcategory_name'])

le_sub = LabelEncoder()
le_sub.fit(train['category_name'])
train['category_name'] = le_sub.transform(train['category_name'])

#### Shopping baskets

In [ ]:
def trp(l, n):
    return [0]*(n-len(l)) + l[:n]

#get baskets
only_bought = train[train['bought'] == 1]
baskets = only_bought.groupby(['basket_hash'], as_index=False).aggregate({'product_id' : 'unique'})
baskets.rename(columns={'product_id' : 'basket'}, inplace=True)
train = pd.merge(train, baskets, how='left', on=['basket_hash'])

basket = pad_sequences(train['basket'],35)
train['basket'] = basket.tolist()

#### Purchase history

In [ ]:
train['day'] = train['day'].astype('str')

In [ ]:
bought_df = train[train['bought'] == 1]
purchase_history = bought_df.groupby(['product_id', 'user_id'], as_index=False).aggregate({'day' : 'unique'})
purchase_history.rename(columns={'day' : 'purchase_history'}, inplace=True)
# merge with actual df
train = pd.merge(train, purchase_history, how='left', on=['user_id', 'product_id'])

user_history = bought_df.groupby(['user_id'], as_index=False).aggregate({'day' : 'unique'})
user_history.rename(columns={'day' : 'user_history'}, inplace=True)
user_history['user_history'] = user_history['user_history'].map(lambda x: np.sort(x))
train = pd.merge(train, user_history, how='left', on=['user_id'])

In [ ]:
train['user_history'] = train['user_history'].map(lambda x: [int(i) for i in x])
train['user_history'] = train['user_history'].map(lambda x: np.sort(x))
train['day'] = train['day'].astype('int')

In [ ]:
def get_last_30(seq, value):
     lst = []
     for s in seq:
         if s < value:
             lst.append(s)
     return lst[-30:]

train['last_30_purchase_days'] = train.apply(lambda x: get_last_30(x.user_history, x.day), axis=1)
train['purchase_history'] = np.where(train['purchase_history'].isna(), "", train['purchase_history'])

In [ ]:
train['purchase_history'] = train['purchase_history'].map(lambda x: [int(i) for i in x])
train['purchase_history'] = train['purchase_history'].map(lambda x: np.sort(x))

train['purchase_history'] = train['purchase_history'].map(lambda x: [str(i) for i in x])
train['last_30_purchase_days'] = train['last_30_purchase_days'].map(lambda x: [str(i) for i in x])


In [ ]:
train['30_day_purchase_history'] = train.apply(lambda x: [1 if str(i) in x.purchase_history else -1 for i in x.last_30_purchase_days], axis=1)

padded_window_30 = pad_sequences(train['30_day_purchase_history'],30,padding='pre')
train['30_day_purchase_history'] = padded_window_30.tolist()

#### Train test split

In [ ]:
last_purchase_day_for_user = train.groupby('user_id', as_index=False).aggregate({'day' : 'max'})
last_purchase_day_for_user.rename(columns=({'day' : 'last_purchase_day'}), inplace=True)

In [ ]:
train = pd.merge(train, last_purchase_day_for_user, how='left', on=['user_id'])

In [ ]:
train['data_split'] = np.where(train['day']==train['last_purchase_day'], 'test', 'train')

In [ ]:
df_train = train[train['data_split'] == 'train']
df_test = train[train['data_split'] == 'test']

#### Price attributes

In [ ]:
df_train['price'] = df_train['price'].astype('float')
df_train['quantity'] = df_train['quantity'].astype('float')

df_test['price'] = df_test['price'].astype('float')
df_test['quantity'] = df_test['quantity'].astype('float')

In [ ]:
# get black prices
train_df_bought = df_train[df_train['bought'] == 1]
black_prices = train_df_bought.groupby('product_id', as_index=False).aggregate({'price' : 'max'})
black_prices.rename(columns=({'price' : 'black_price'}), inplace=True)

df_train = pd.merge(df_train, black_prices, how='left', on=['product_id'])

df_train['price'] = df_train['price'].astype('float')
df_train['discount'] = (df_train['black_price'] -  df_train['price'])/df_train['black_price']

In [ ]:
# get black price to test data and calculate discount
df_test = pd.merge(df_test, black_prices, how='left', on=['product_id'])
df_test['price'] = df_test['price'].astype('float')
df_test['discount'] = (df_test['black_price'] -  df_test['price'])/df_test['black_price']

In [ ]:
df_train['discount'].fillna(0.000000, inplace=True)
df_test['discount'].fillna(0.000000, inplace=True)

#### Other products in the basket

In [ ]:
#get other products in the basket
only_bought = df_train[df_train['bought'] == 1]
baskets = only_bought.groupby(['basket_hash'], as_index=False).aggregate({'product_id' : 'unique'})
baskets.rename(columns={'product_id' : 'basket'}, inplace=True)
only_bought = pd.merge(only_bought, baskets, how='left', on=['basket_hash'])
only_bought['other_prods_in_basket'] = only_bought.apply(lambda x: [i for i in x.basket if i != x.product_id], axis=1)
only_bought['product_index'] = only_bought['product_index'].astype('str')
only_bought['basket_hash'] = only_bought['basket_hash'].astype('str')
only_bought['basket_index'] = only_bought[['basket_hash', 'product_index']].agg('_'.join, axis=1)
only_bought_basket_info = only_bought[['other_prods_in_basket', 'basket_index']]
df_train['product_index'] = df_train['product_index'].astype('str')
df_train['basket_hash'] = df_train['basket_hash'].astype('str')
df_train['basket_index'] = df_train[['basket_hash', 'product_index']].agg('_'.join, axis=1)
df_train = pd.merge(df_train, only_bought_basket_info, how='left', on=['basket_index'])

In [ ]:
#get other products in the basket for test data
only_bought = df_test[df_test['bought'] == 1]
baskets = only_bought.groupby(['basket_hash'], as_index=False).aggregate({'product_id' : 'unique'})
baskets.rename(columns={'product_id' : 'basket'}, inplace=True)
only_bought = pd.merge(only_bought, baskets, how='left', on=['basket_hash'])
only_bought['other_prods_in_basket'] = only_bought.apply(lambda x: [i for i in x.basket if i != x.product_id], axis=1)
only_bought['product_index'] = only_bought['product_index'].astype('str')
only_bought['basket_hash'] = only_bought['basket_hash'].astype('str')
only_bought['basket_index'] = only_bought[['basket_hash', 'product_index']].agg('_'.join, axis=1)
only_bought_basket_info = only_bought[['other_prods_in_basket', 'basket_index']]
df_test['product_index'] = df_test['product_index'].astype('str')
df_test['basket_hash'] = df_test['basket_hash'].astype('str')
df_test['basket_index'] = df_test[['basket_hash', 'product_index']].agg('_'.join, axis=1)
df_test = pd.merge(df_test, only_bought_basket_info, how='left', on=['basket_index'])

In [ ]:
df_test['other_prods_in_basket'] = df_test.other_prods_in_basket.apply(lambda x: np.array(x, dtype=np.int))
df_train['other_prods_in_basket'] = df_train.other_prods_in_basket.apply(lambda x: np.array(x, dtype=np.int))

In [ ]:
def trp(l, n):
    return [0]*(n-len(l)) + l[:n]

df_train['other_prods_in_basket'] = df_train.other_prods_in_basket.apply(lambda x: trp(x, 35))
df_test['other_prods_in_basket'] = df_test.other_prods_in_basket.apply(lambda x: trp(x, 35))

#### Sample weights

In [ ]:
df_bought = df_train[df_train['bought'] == 1]
df_bought['total_purchases'] = df_bought.groupby('product_id')['product_id'].transform('count')
product_frequency = df_bought[['product_id', 'total_purchases']].drop_duplicates()
df_train = pd.merge(df_train, product_frequency, on=['product_id'], how='left')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
min_max = MinMaxScaler()

In [ ]:
for_not_bought = min_max.fit_transform(pd.DataFrame(df_train['total_purchases']))
df_train['for_not_bought'] = for_not_bought

In [ ]:
df_train['for_bought'] = 1 / df_train['total_purchases']
for_bought = min_max.fit_transform(pd.DataFrame(df_train['for_bought']))
df_train['for_bought'] = for_bought

In [ ]:
df_train['sample_weight'] = np.where(df_train['bought'] == 1, df_train['for_bought'], df_train['for_not_bought'])

We will use the calculated sample weights for train data for test data as well.

In [ ]:
train_weights = df_train[['product_id', 'total_purchases', 'for_bought', 'for_not_bought']]
train_weights = train_weights.drop_duplicates()
df_test = pd.merge(df_test, train_weights, how='left', on=['product_id'])
df_test['sample_weight'] = np.where(df_test['bought'] == 1, df_test['for_bought'], df_test['for_not_bought'])

#### Difference in discounts per substitute products

In [ ]:
df_train['diffs'] = df_train.groupby(['product_index'])['discount'].transform(lambda x: x.diff())
df_train["diffs"] = df_train.groupby("product_index")['diffs'].transform(lambda x: x.fillna(x.mean()*-1))

In [ ]:
df_test['diffs'] = df_test.groupby(['product_index'])['discount'].transform(lambda x: x.diff())
df_test["diffs"] = df_test.groupby("product_index")['diffs'].transform(lambda x: x.fillna(x.mean()*-1))

#### Save data as a .pkl file

In [ ]:
#merge test and train data
df_train['data_split'] = 'train'
df_test['data_split'] = 'test'

df = df_train.append(df_test, ignore_index=True)

del df['purchase_history'], df['user_history'], df['last_30_purchase_days'], df['last_purchase_day'], df['black_price'],df['basket_index'],df['for_not_bought'],df['for_bought']     

In [ ]:
with open('data_preprocessed.pkl', 'wb') as handle:
    pickle.dump(df, handle)

from google.colab import files
files.download('data_preprocessed.pkl')